In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
import xgboost as xgb
from sklearn.linear_model import LogisticRegression

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/school-21-student-expulsion-prediction/train.csv
/kaggle/input/school-21-student-expulsion-prediction/test.csv
/kaggle/input/school-21-student-expulsion-prediction/waves_dates.csv
/kaggle/input/school-21-student-expulsion-prediction/sample_submission.csv


In [2]:
train = pd.read_csv("/kaggle/input/school-21-student-expulsion-prediction/train.csv")
test = pd.read_csv("/kaggle/input/school-21-student-expulsion-prediction/test.csv")
answer = pd.read_csv("/kaggle/input/school-21-student-expulsion-prediction/test.csv")

In [3]:
train.head()

,id,Birth date,Native city,Gender,Wave id,Level,Heard about school from,Life status,day_00,day_01,...,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Contract termination date,Memory entrance game,Logic entrance game,contract_status
0,129212391,1990-01-18,Пермь,male,3,7.463235,google_ads,work,5,NaN,...,0.0,0.0,75.0,43.0,100.0,44.0,NaN,9,11,1
1,566688420,1996-07-31,Волгоград,female,1,8.785714,NaN,NaN,0,NaN,...,0.0,0.0,0.0,16.0,22.0,22.0,NaN,7,10,1
2,242300495,1992-06-27,Новосибирск,female,2,6.775000,other,work,0,NaN,...,NaN,NaN,0.0,0.0,16.0,27.0,NaN,7,11,1
3,715424753,1996-06-15,Тула,female,1,8.146104,NaN,NaN,5,0.0,...,0.0,0.0,0.0,0.0,27.0,27.0,NaN,9,6,1
4,382884118,1968-07-18,Голицыно,male,3,4.272727,other,other,0,0.0,...,0.0,0.0,80.0,38.0,0.0,45.0,NaN,7,12,1


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1060 entries, 0 to 1059
Data columns (total 36 columns):
id                           1060 non-null int64
Birth date                   1060 non-null object
Native city                  1047 non-null object
Gender                       1060 non-null object
Wave id                      1060 non-null int64
Level                        1060 non-null float64
Heard about school from      653 non-null object
Life status                  655 non-null object
day_00                       1060 non-null int64
day_01                       646 non-null float64
day_02                       1044 non-null float64
day_03                       1045 non-null float64
day_04                       1037 non-null float64
day_05                       1056 non-null float64
day_06                       1048 non-null float64
day_07                       1046 non-null float64
day_08                       1045 non-null float64
day_09                       1047 non-nu

1. All columns day count avg
2. Same with rush
3. Same with exam
3. Delete match bsq eval heard id 

In [5]:
#удаляем ненужные фичи
train = train.drop(["id", "Heard about school from","Memory entrance game", "Contract termination date"], axis = 1)
test = test.drop(["id","Heard about school from","Memory entrance game", "Contract termination date"], axis = 1)

In [6]:
#count age
# new data frame with split value columns 
new = train["Birth date"].str.split("-", n = 1, expand = True)
new1 = test["Birth date"].str.split("-", n = 1, expand = True)
  
# making separate first name column from new data frame 
train["age"] = new[0]
test["age"] = new[0]

#delete birth date
train = train.drop("Birth date", axis = 1)
test = test.drop("Birth date", axis = 1)

In [7]:
train["age"].unique()

array(['1990', '1996', '1992', '1968', '2000', '1999', '1997', '1991',
       '1987', '1998', '1993', '1988', '1985', '1995', '2001', '1981',
       '1994', '1986', '1989', '1979', '1983', '1984', '1980', '1959',
       '1976', '1977', '1967', '1978', '1973', '1964', '1963', '1965',
       '1971', '1974', '1957', '1982'], dtype=object)

In [8]:
test["age"].unique()

array(['1990', '1996', '1992', '1968', '2000', '1999', '1997', '1991',
       '1987', '1998', '1993', '1988', '1985', '1995', '2001', '1981',
       '1994', '1986', '1989', '1979', '1983', '1984', '1980', '1959',
       '1976', '1977', '1967', '1978', '1973', '1964', '1963', '1965',
       '1971', '1974', '1957'], dtype=object)

In [9]:
#binning age before 1975 and after
bin1_bef75_train = ['1968', '1959','1967', '1973', '1964', '1963', '1965','1971', '1974', '1957', '1968']
bin2_af75_train = [ '1987', '1988', '1985', '1981', '1986', '1989', '1979', '1983', '1984', '1980', '1976', '1977',  '1978','1982']
bin3_af90_train = ['1990', '1996', '1992', '2000','1999', '1997', '1991', '1994', '1995', '2001', '1998', '1993']


bin1_bef75_test = ['1968','1959','1967', '1973', '1964', '1963', '1965','1971', '1974', '1957']
bin2_af75_test = ['1987', '1988', '1985', '1981', '1986', '1989', '1979', '1983', '1984', '1980','1976', '1977', '1978']
bin3_af90_test =['1990', '1996', '1992', '2000', '1999', '1997', '1991', '1998', '1993','1994', '1995', '2001']

train["age"] = train["age"].replace(bin1_bef75_train, 0)
train["age"] = train["age"].replace(bin2_af75_train, 1)
train["age"] = train["age"].replace(bin3_af90_train, 2)

test["age"] = test["age"].replace(bin1_bef75_test, 0)
test["age"] = test["age"].replace(bin2_af75_train, 1)
test["age"] = test["age"].replace(bin3_af90_train, 2)

In [10]:
#work with life status
test["Life status"] = test["Life status"].fillna(0)
test["Life status"] = test["Life status"].replace("work", 1)
test["Life status"] = test["Life status"].replace("study", 2)
test["Life status"] = test["Life status"].replace("other", 0)

train["Life status"] = train["Life status"].fillna(0)
train["Life status"] = train["Life status"].replace("work", 1)
train["Life status"] = train["Life status"].replace("study", 2)
train["Life status"] = train["Life status"].replace("other", 0)

In [11]:
#encoding gender
train["Gender"] = train["Gender"].replace("male", 0)
train["Gender"] = train["Gender"].replace("female", 1)

test["Gender"] = test["Gender"].replace("male", 0)
test["Gender"] = test["Gender"].replace("female", 1)

In [12]:
test["Native city"] = test["Native city"].fillna(0)
test["Native city"] = test["Native city"].replace(["Голицыно", "Moscow", "Москва", "Долгопрудный ", "Егорьевск",
                                                   "Кашира", "Лосино-Петровский", "Одинцовский район, поселок Лесной городок",
                                                   "Реутов", "Химки", "Дер. Щемилово, Ногинский р-н", "дер. Сапроново", "гор. Мытищи", "Зеленоград", 
                                                   "Москва ", "Дубна", "Перхушково", "деревня Дрожжино", "Жуковский", "МО, Истринский район, д.Черная", "Долгопрудный",
                                                   "Королёв", "с. Кудиново", "д. Ледово ", "Электросталь", "Мытищи", "Москве", "Zheleznodorozhniy", "Московский", "Железнодорожный", 
                                                   "Краснознаменск", "Лобня", "Пос. Внуковское д. Рассказовка ", "Юбилейный", "Звенигород", "Солнечногорск", "Подольск", "Красногорск", 
                                                   "Дзержинский", "деревня Сапроново", " Москва", "пос. Калининец", "Рп. Ильинский", "Молоковское", "Щелково", "р.п.Ржавки", "г. Домодедово",
                                                   "Люберцы", "г. Ликино-Дулево", "г. Москва", "С.Красное", "Москвп", "ДП Удельная", "Сергиев Посад", "Одинцово", "Mosocow", "Пушкино", "Старая Купавна", 
                                                   "Серпухов", "Dolgoprudny", "п. Первомайский", "Троицк", "г. Москва, ", "пос. Первомайский", "Чехов", "Балашиха, мкр. Железнодорожный", "Пущино", "С. Подхожее", 
                                                   "Климовск", "Лыткарино", "Moscow region, Odintsovo", "Видное", "п. Нахабино", "Чеховский район, с. Троицкое", "Котельники", "Фрязино", "Lubercy", "г. Котельники", 
                                                   "Московская область", "Дмитров", "Десеновское поселение", "Московская обл. г. Мытищи", "дер. Митькино", "Быково", "дер. Кузнецово", "Московская обл. Чеховский р-н, СНТ Снежная долина 2",
                                                   "Снегири", "пос. Сосенское", "Krasnogorsk", "Королев", "г. Одинцово", "Раменское", "гор. Балашиха ", "с. Молоково", "Podolsk", "Барвиха", "Pushkino", "город Москва", "Мытищи ", 
                                                   "г. Железнодорожный", "г. Химки", "поселок ВНИИСОК", "пос. Октябрьский",
                                                   "п. Десеновское", "Черноголовка", "рп Новоивановское", "пос. ВНИИССОК", "Р-Н ВОСКРЕСЕНСКИЙ, НП СЛОБОДКА АЛЕШИНО", 
                                                   "Ногинск", "Домодедово", "д. Мисайлово", "Протвино", "Красногорск ", "Балашиха, мкр Железнодорожный", "Хотьково", 
                                                   "Balashikha", "пос. Красково", "г. Раменское", "Андреевка", "дер. Петелино", "КРАСНОГОРСК", "дер. Пучково", "Рп Шаховская", "д. Мизиново", "Можайск", "Нахабино", "гп.Калининц", "Щелковский район, деревня Ледово", "Зеленоград", "москва", "Ивантеевка", "Клин пос. Майданово", "С. Рождествено ", "Истра", "Видное ", "Мытищи, дер. Еремино", "МОСКВА", "Lubertsy", "Ramenskii raion", 
                                                   "Воскресенск", "Лыткарино ", "Коломна", "с. Петровское", "Солнечногорск ", "Odintsovo", "красногорск ", "Малые Вязёмы"], 0)
test["Native city"] = test["Native city"].replace([ "г. Тюмень", "Omsk", "Благовещенск", "Velikie Luki", "Белгород", "Санкт Петербург", "с. Аскарово", "Сельцо", "п. Айхал", "гор. Екатеринбург", "с. Тесово", "Sankt Peterburg", "Ставропольский край, г.Ставрополь", "Каневская", "Орел", "Ершов", "Шелеховский район, с. Баклаши", "Кумертау", "Выкса", "Санкт-Петербург, п. Парголово", "Ст. Старосышастовская ", "Стерлитамак", "Таганрог", "Набережные Челны",
                                                   "пос. Бавлены", "с. Молочное", "Псков", "Рязань ", "Анапа", "Комосомольск-на-Амуре", "Ростов на Дону ", "Магнитогорск", "Мурманск", 
                                                   "Ростов-на-дону", "Полевской", "г. Пенза", "Уразаево", "Касимов", "Новодвинск", "Новосибирская область, Новосибирский район, с. Толмачево, мкр.", "Пригородный простор", "Сысерть", "Донской", "г. Новый Уренгой", "Калининград", "Тамбов", "Прионежский р-н, п. Шуя", "Караганда", "Дзержинск", "Пенза", "гор. Калуга", "Чайковский", "с. Ближняя Игуменка", "Соликамск", "п. Сурок", 
                                                   "С. Писцово", "Керчь", "п. Юго-Камский ", "Фролово", "Белоусово", "Жлобин", "п. Тартазяк", "г. Москва, поселение Московский, гор. Московский", "Нововоронеж", "Kirov", "Михайловка", "с. Миасское", "с. Ястребовка", "ЧУВАШСКАЯ РЕСПУБЛИКА, ЧЕБОКСАРЫ",
                                                   "Дмитриев", "гор. Славгород", "Кострома", "Петропавловск-Камчатский", "Качканар", "Октябрьск", "ALMATY", "Сахалинск", "Братск", "Павловский посад", "Переславль-Залеский", "г. Нововоронеж", "пгт Аксубаево",
                                                   "Заполярный", "Всеволожск", "Донецк", "Чудово", "СПб", "Самарская обл., г. Самара", "Асбест", "Sochi", "Тихорецк", "С. Набережные Моркваши, Верхнеуслонский район", "с. Подклетное", "Каменск-Шахтинский", "Saint-Petersburg", "Сорочинск", "с. Каплино", "Красная Гора ", "Ульяновск ", "С.Татарский шуган", "Воркута ", "Феодосия", "пос. Шушары" , "Пермь", "Волгоград", "Новосибирск", "Тула", "Мичуринск", "Вологда",
                                                   "пос. Майский", "Новороссийск", "Krasnoyarsk", "Брянск", "Торжок", "Cheboksary", "Курск", "Рязань", "Набережные Челны ", 
                                                   "Нижний Новгород", "Челябинск", "Бор", "Батайск", "Дмитровск", "Киров", "Санкт-Петербург", "Нягань", "Чебоксары", "Обнинск", "Якутск",
                                                   "Новочебоксарск", "Красноярск", "Омск", "Республика Дагестан, г. Махачкала", "Саратов", "Северодвинск", "с. Учкекен", "Сыктывкар", "Магас", "Уфа", 
                                                   "Ижевск", "с. Виноградное", "р.п. Панино", "Балашиха", "п. Вернетемерницкий", "Краснодар", "г. Новоаннинский", "Липецк", "Радужный ", "Иркутск",
                                                   "р-н Кимрски, дер. Абрамово", "Самара", "Екатеринбург", "Хабаровск", "Владивосток", "Оренбург", "Ставрополь", "Казань", "Кириши", "д. Кяростров", 
                                                   "Павлово", "Чернь", "Малгобек", "Кулебаки", "Добрянка", "РП. ИНЖАВИНО", "Томск", "Сургут", "Северск", "Минеральные Воды", "пос. Пруды", "Архангельск", 
                                                   "Йошкар-Ола", "Новочеркасск", "Ковров", "Кольчугино", "Ростов-на-Дону", "Ош", "Альметьевск", "Великий Новгород", "Дер.Варечки", "Черемхово", "г. Энгельс",
                                                   "Сочи", "с. Георгиевка", "Ульяновск", "Воронеж", "Bryansk", "Город Брянск", "Буденновск", "Новошахтинск", "пос. Черноморское", "Тольятти", "Южно-Сахалинск",
                                                   "Saint Petersburg", "ст. Полтавская", "с. Терновое", "Село Верхние Татышлы", "Пятигорск", "Новокузнецк", "г. Санкт -Петербург", "Малоярославец", "Азнакаево",
                                                   "Рязанская обл. Скопинский р-н с. Ермолово", "Тобольск", "Калуга", "Назрань", "Владикавказ", "Чебоксары ", "Михайловск", "Шелехов", "Наро-Фоминск", "Петрозаводск",
                                                   "РП Кормиловка", "рп Решетиха", "Чита", "Прокопьевск", "Мценск", "С.Бурнак", "п. Новосадовый", "Нефтекамск", "Лебедянь", "Вязьма", "Ярославль", "Дзержинск ", "Орёл",
                                                   "Тюмень", "Улан-Удэ", "Сосновый", "Чадан", "Волжский", "Элиста ", "Дер.Кудрово", "Нижняя Мара", "гор. Иркутск", "Воткинск ", "Гуково", "Каа-Хем", "Иваново", "Ангарск",
                                                   "с. Чирон", "Узловской район, пос. Краснолесский", "Ачинск", "г.Пермь", "Нальчик", "Владимир", "г. Котово", "Удомля", "Новомосковск", "Краснодар ", "Таганрог ", "Череповец",
                                                   "Нефтеюганск", "гор. Острогожск", "Ханты-Мансийский автономный округ, г. Нефтеюганск", "Каспийск", "Майкоп", "Новоуральск", "п. Цементный", "п. Пристень", "Irkutsk", "пгт. Забайкальск",
                                                   "Саранск", "Муравленко", "Шацк", "с. Грязное", "Тверская обл., г. Нелидово ", "Almaty", "Шымкент", "п.Визимьяры", "с. Михайловка", "Санкт-петербург", "Чувашия, Моргаушский район", "Ставропольский край, г. Светлоград",
                                                   "Тверь", "ст-ца Тацинская", "П.Горняцкий", "Орск", "Шумерля", "пос. Визимьяры", "КОГАЛЫМ", "Tymen", " ст.Нововеличковская", "Бурск", "ст. Ессентукская", "Козьмодемьянск", "Копейск", 
                                                   "с. Лащ-Таяба", "Смоленск", "Балаково", "Конаково", "Гулькевичи", "пгт. Каа-хем", "Крымс", "Симферополь", "Армавир", "Курган", "пгт Красное-на-Волге", "Вятские Поляны", "Махачкала", "Новозыбков", "Жигулевск",
                                                   "С. Подлесное", "г. Инза", "Вилючинск", "Тарко-Сам", "Санкт-Петербург ", "Дер. Нурма", "с. Сонское", "п. Московский Тюменского района", "Арамиль", 'Новосибирская область, Новосибирский район, с. Толмачево, мкр. "Пригородный простор"',
       'Зеленоград '], 1)

In [13]:
train["Native city"] = train["Native city"].fillna(0)
train["Native city"] = train["Native city"].replace(["Голицыно", "Moscow", "Москва", "Долгопрудный ", "Егорьевск",
                                                   "Кашира", "Лосино-Петровский", "Одинцовский район, поселок Лесной городок",
                                                   "Реутов", "Химки", "Дер. Щемилово, Ногинский р-н", "дер. Сапроново", "гор. Мытищи", "Зеленоград", 
                                                   "Москва ", "Дубна", "Перхушково", "деревня Дрожжино", "Жуковский", "МО, Истринский район, д.Черная", "Долгопрудный",
                                                   "Королёв", "с. Кудиново", "д. Ледово ", "Электросталь", "Мытищи", "Москве", "Zheleznodorozhniy", "Московский", "Железнодорожный", 
                                                   "Краснознаменск", "Лобня", "Пос. Внуковское д. Рассказовка ", "Юбилейный", "Звенигород", "Солнечногорск", "Подольск", "Красногорск", 
                                                   "Дзержинский", "деревня Сапроново", " Москва", "пос. Калининец", "Рп. Ильинский", "Молоковское", "Щелково", "р.п.Ржавки", "г. Домодедово",
                                                   "Люберцы", "г. Ликино-Дулево", "г. Москва", "С.Красное", "Москвп", "ДП Удельная", "Сергиев Посад", "Одинцово", "Mosocow", "Пушкино", "Старая Купавна", 
                                                   "Серпухов", "Dolgoprudny", "п. Первомайский", "Троицк", "г. Москва, ", "пос. Первомайский", "Чехов", "Балашиха, мкр. Железнодорожный", "Пущино", "С. Подхожее", 
                                                   "Климовск", "Лыткарино", "Moscow region, Odintsovo", "Видное", "п. Нахабино", "Чеховский район, с. Троицкое", "Котельники", "Фрязино", "Lubercy", "г. Котельники", 
                                                   "Московская область", "Дмитров", "Десеновское поселение", "Московская обл. г. Мытищи", "дер. Митькино", "Быково", "дер. Кузнецово", "Московская обл. Чеховский р-н, СНТ Снежная долина 2",
                                                   "Снегири", "пос. Сосенское", "Krasnogorsk", "Королев", "г. Одинцово", "Раменское", "гор. Балашиха ", "с. Молоково", "Podolsk", "Барвиха", "Pushkino", "город Москва", "Мытищи ", 
                                                   "г. Железнодорожный", "г. Химки", "поселок ВНИИСОК", "пос. Октябрьский",
                                                   "п. Десеновское", "Черноголовка", "рп Новоивановское", "пос. ВНИИССОК", "Р-Н ВОСКРЕСЕНСКИЙ, НП СЛОБОДКА АЛЕШИНО", 'пос.  Первомайский'], 0)
train["Native city"] = train["Native city"].replace([ "Пермь", "Волгоград", "Новосибирск", "Тула", "Мичуринск", "Вологда",
                                                   "пос. Майский", "Новороссийск", "Krasnoyarsk", "Брянск", "Торжок", "Cheboksary", "Курск", "Рязань", "Набережные Челны ", 
                                                   "Нижний Новгород", "Челябинск", "Бор", "Батайск", "Дмитровск", "Киров", "Санкт-Петербург", "Нягань", "Чебоксары", "Обнинск", "Якутск",
                                                   "Новочебоксарск", "Красноярск", "Омск", "Республика Дагестан, г. Махачкала", "Саратов", "Северодвинск", "с. Учкекен", "Сыктывкар", "Магас", "Уфа", 
                                                   "Ижевск", "с. Виноградное", "р.п. Панино", "Балашиха", "п. Вернетемерницкий", "Краснодар", "г. Новоаннинский", "Липецк", "Радужный ", "Иркутск",
                                                   "р-н Кимрски, дер. Абрамово", "Самара", "Екатеринбург", "Хабаровск", "Владивосток", "Оренбург", "Ставрополь", "Казань", "Кириши", "д. Кяростров", 
                                                   "Павлово", "Чернь", "Малгобек", "Кулебаки", "Добрянка", "РП. ИНЖАВИНО", "Томск", "Сургут", "Северск", "Минеральные Воды", "пос. Пруды", "Архангельск", 
                                                   "Йошкар-Ола", "Новочеркасск", "Ковров", "Кольчугино", "Ростов-на-Дону", "Ош", "Альметьевск", "Великий Новгород", "Дер.Варечки", "Черемхово", "г. Энгельс",
                                                   "Сочи", "с. Георгиевка", "Ульяновск", "Воронеж", "Bryansk", "Город Брянск", "Буденновск", "Новошахтинск", "пос. Черноморское", "Тольятти", "Южно-Сахалинск",
                                                   "Saint Petersburg", "ст. Полтавская", "с. Терновое", "Село Верхние Татышлы", "Пятигорск", "Новокузнецк", "г. Санкт -Петербург", "Малоярославец", "Азнакаево",
                                                   "Рязанская обл. Скопинский р-н с. Ермолово", "Тобольск", "Калуга", "Назрань", "Владикавказ", "Чебоксары ", "Михайловск", "Шелехов", "Наро-Фоминск", "Петрозаводск",
                                                   "РП Кормиловка", "рп Решетиха", "Чита", "Прокопьевск", "Мценск", "С.Бурнак", "п. Новосадовый", "Нефтекамск", "Лебедянь", "Вязьма", "Ярославль", "Дзержинск ", "Орёл",
                                                   "Тюмень", "Улан-Удэ", "Сосновый", "Чадан", "Волжский", "Элиста ", "Дер.Кудрово", "Нижняя Мара", "гор. Иркутск", "Воткинск ", "Гуково", "Каа-Хем", "Иваново", "Ангарск",
                                                   "с. Чирон", "Узловской район, пос. Краснолесский", "Ачинск", "г.Пермь", "Нальчик", "Владимир", "г. Котово", "Удомля", "Новомосковск", "Краснодар ", "Таганрог ", "Череповец",
                                                   "Нефтеюганск", "гор. Острогожск", "Ханты-Мансийский автономный округ, г. Нефтеюганск", "Каспийск", "Майкоп", "Новоуральск", "п. Цементный", "п. Пристень", "Irkutsk", "пгт. Забайкальск",
                                                   "Саранск", "Муравленко", "Шацк", "с. Грязное", "Тверская обл., г. Нелидово ", "Almaty", "Шымкент", "п.Визимьяры", "с. Михайловка", "Санкт-петербург", "Чувашия, Моргаушский район", "Ставропольский край, г. Светлоград",
                                                   "Тверь", "ст-ца Тацинская", "П.Горняцкий", "Орск", "Шумерля", "пос. Визимьяры", "КОГАЛЫМ", "Tymen", " ст.Нововеличковская", "Бурск", "ст. Ессентукская", "Козьмодемьянск", "Копейск", 
                                                   "с. Лащ-Таяба", "Смоленск", "Балаково", "Конаково", "Гулькевичи", "пгт. Каа-хем", "Крымс", "Симферополь", "Армавир", "Курган", "пгт Красное-на-Волге", "Вятские Поляны", "Махачкала", "Новозыбков", "Жигулевск",
                                                   "С. Подлесное", "г. Инза", "Вилючинск", "Тарко-Сам", "Санкт-Петербург ", "Дер. Нурма", "с. Сонское", "п. Московский Тюменского района", "Арамиль"], 1)

In [14]:
train = train.fillna(0)
test = test.fillna(0)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1060 entries, 0 to 1059
Data columns (total 32 columns):
Native city            1060 non-null int64
Gender                 1060 non-null int64
Wave id                1060 non-null int64
Level                  1060 non-null float64
Life status            1060 non-null int64
day_00                 1060 non-null int64
day_01                 1060 non-null float64
day_02                 1060 non-null float64
day_03                 1060 non-null float64
day_04                 1060 non-null float64
day_05                 1060 non-null float64
day_06                 1060 non-null float64
day_07                 1060 non-null float64
day_08                 1060 non-null float64
day_09                 1060 non-null float64
day_10                 1060 non-null float64
day_11                 1060 non-null float64
day_12                 1060 non-null float64
day_13                 1060 non-null float64
evalexpr               1060 non-null float64
mat

In [16]:
#split x, y
X_train = train.drop("contract_status", axis = 1)
Y_train = train["contract_status"]

In [17]:
model = xgb.XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.1, gamma=0.0,
              gpu_id=-1, importance_type='gain', interaction_constraints=None,
              learning_rate=0.100000001, max_delta_step=0, max_depth=3,
              min_child_weight=1, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None).fit(X_train, Y_train)

In [18]:
lr = LogisticRegression(max_iter = 1000)
lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
answer["contract_status"] = lr.predict_proba(test)[:,1]
answer = answer[["id","contract_status"]]
answer.to_csv('submission.csv', index = False)